In [ ]:
#idk diference between cv and cv2. I think cv2 has class functionality
#import cv

In [1]:
#library needed on beaglebone
import cv2
import numpy as np
import time

In [2]:
import matplotlib
import matplotlib.pyplot as plt

In [3]:
def get_480p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def get_cut_480p(file):
    img=cv2.imread(file)
    img1=img[320:,:]
    gray_img=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    return img1,gray_img
def get_360p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(320,240))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def display(img):
    try:
        b,g,r = cv2.split(img)
        img2 = cv2.merge([r,g,b])
        imgplot = plt.imshow(img)
        plt.show()
    except:    
        imgplot = plt.imshow(img,cmap='gray')
        plt.show()
def hough_trans(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0,plot_img=0):
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
                display(edges)
    #print lines
    if plot_img==1:
        try:
            for rho,theta in lines[0]:
                print rho,theta
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
                cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_transp(img,gray,can_min=150,can_max=300,vote=150,
                 minLineLength = 20,maxLineGap = 15,plot_canny=0,plot_img=0):
    #hough that uses less test points, less computations
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLinesP(edges,1,np.pi/180,vote,minLineLength,maxLineGap)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    if plot_canny==1:
        display(edges)
    #print lines
    if plot_img==1:
        try:
            for x1,y1,x2,y2 in lines[0]:
                cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
            display(img)
        except:
            print('could not plot')
    else:
        try:
            return len(lines[0])
        except:
            return 0
def hough_trans_blur(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0):
    t1 = time.time()
    blur = cv2.blur(gray,(2,2))
    t2 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t3 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t4 = time.time()
    print 'blur', t2 - t1
    print 'canny',t3 - t2
    print 'hough',t4 - t3
    #print lines
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    if plot_canny==1:
        display(edges)
    display(img)

In [ ]:
img1,gray=get_480p('2-knee.jpg')
display(img1)
#blur = cv2.bilateralFilter(gray,9,75,75)
#display(blur)
hough_trans(img1,gray,vote=220,can_min=150,can_max=250)

In [8]:
#iterating hough
img1,gray=get_480p('1-a.jpg')
#img1,gray=get_360p('2-knee.jpg')
display(img1)
it_vote=200
lvote=[it_vote]
lln=[]
mu=.75
n_lines=25
for x in range(0,20):
    ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

canny 0.0159690380096
hough 0.0594630241394
canny 0.00412583351135
hough 0.039165019989
canny 0.00395011901855
hough 0.0327048301697
canny 0.00391983985901
hough 0.0439879894257
canny 0.00376510620117
hough 0.031387090683
canny 0.0038731098175
hough 0.0439279079437
canny 0.00368499755859
hough 0.0306220054626
canny 0.00390386581421
hough 0.034784078598
canny 0.00378108024597
hough 0.0309338569641
canny 0.00367498397827
hough 0.0439021587372
canny 0.00378179550171
hough 0.0324721336365
canny 0.00373196601868
hough 0.0437178611755
canny 0.00358510017395
hough 0.0315179824829
canny 0.00372219085693
hough 0.0437188148499
canny 0.00405406951904
hough 0.0314509868622
canny 0.00389504432678
hough 0.0443139076233
canny 0.00359606742859
hough 0.031476020813
canny 0.00388598442078
hough 0.0446491241455
canny 0.00377607345581
hough 0.0326941013336
canny 0.00389003753662
hough 0.0436959266663
canny 0.00379800796509
hough 0.0311269760132
157.0 1.65806
153.0 1.65806
488.0 0.890118
494.0 0.872665
500

In [14]:
img,gray=get_cut_480p('4-knee.jpg')
display(img)
it_vote=200
lvote=[it_vote]
lln=[]
mu=.45
n_lines=2
for x in range(0,20):
    ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250)
    lln.append(ln)
    fixed=n_lines-ln
    #iterate
    it_vote=it_vote-mu*fixed
    lvote.append(it_vote)
ln=hough_trans(img1,gray,vote=int(it_vote),can_min=200,can_max=250,plot_canny=1,plot_img=1)
print(lvote)
plt.figure(1)
plt.plot(lvote)
plt.ylabel('Vote')
plt.xlabel('Iteration')
plt.figure(2)
plt.plot(lln)
plt.ylabel('Number of lines')
plt.xlabel('Iteration')
plt.show()

canny 0.0276520252228
hough 0.111211061478
canny 0.0162751674652
hough 0.102469921112
canny 0.0204839706421
hough 0.10874080658
canny 0.0201292037964
hough 0.104791879654
canny 0.0171658992767
hough 0.103147029877
canny 0.0166850090027
hough 0.1023209095
canny 0.0170180797577
hough 0.102043867111
canny 0.0168590545654
hough 0.102880954742
canny 0.0166149139404
hough 0.109311103821
canny 0.0163550376892
hough 0.101863861084
canny 0.0157868862152
hough 0.103569030762
canny 0.0160369873047
hough 0.114208221436
canny 0.016685962677
hough 0.112518072128
canny 0.0167601108551
hough 0.104924917221
canny 0.016802072525
hough 0.103017807007
canny 0.0168080329895
hough 0.103849887848
canny 0.016487121582
hough 0.103917837143
canny 0.0166800022125
hough 0.102876901627
canny 0.0167369842529
hough 0.105627059937
canny 0.0167851448059
hough 0.102952003479
canny 0.0166349411011
hough 0.105222940445
could not plot
[200, 1334.45, 1333.55, 1332.6499999999999, 1331.7499999999998, 1330.8499999999997, 1329

In [21]:
#circles
img,gray=get_480p('1-a.jpg')
display(gray)
blur = cv2.medianBlur(gray,5)
display(blur)
circles = cv2.HoughCircles(blur,3,30,3)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(gray,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(gray,(i[0],i[1]),2,(0,0,255),3)
display(gray)

In [ ]:
#get low image resolution or take midsection of high res.
#blur, use the bit operator to only output where the sidewalk is, travel on right side.
#in forest area use the high intensity light as a sign of the sidewalk being there
#perhaps make algorithim to detect white

In [ ]:
#try to use the averaging filters to get the whole concrete slab
img,gray=get_480p('8.jpg')
blur = cv2.blur(gray,(2,2))
display(blur)
hough_trans(img,gray)

In [ ]:
#use bilateral filtering to remove texture(concrete) but is cpu intensive. 
img,gray=get_480p('8.jpg')
blur = cv2.bilateralFilter(gray,9,75,75)
display(blur)
hough_trans(img,gray)

In [ ]:
#individual colors
img,gray=get_480p('4-knee.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_blue = np.array([50,50,110])
upper_blue = np.array([255,255,130])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
 # Bitwise-AND mask and original image
res = cv2.bitwise_and(img,img, mask= mask)
display(img)
display(mask)
display(res)

In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img=cv2.imread('4-knee.jpg')

In [ ]:
#returns (height, width, pixel element number)
height, width, pixel=img.shape
print img.shape

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img2 = img[280:340, 330:390]#take section of the picture
img[273:333, 100:160] = img2#put that section on original picutre

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
b,g,r = cv2.split(img)#full splitting into colors
b = img[:,:,0]#individual blue color
imgplot = plt.imshow(b)
plt.show()

In [ ]:
#Grayscale
gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgplot = plt.imshow(gray_img)
plt.show()

In [ ]:
#HSV
hsv_img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
imgplot = plt.imshow(hsv_img)
plt.show()

In [ ]:
print img[0,0]
print gray_img[0,0]
print hsv_img[0,0]

In [ ]:
#capture video
cap = cv2.VideoCapture(0)
while 1:
    ret, frame = cap.read()
    if ret==True:#check if cap.read() returned frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        display(gray)
        #image recognition code here
    break

In [ ]:
#canny edge detection
img = cv2.imread('4-knee.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
img = cv2.imread('1-a.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,160,300,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)
#print lines
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
#plt.imshow(img,cmap='gray')
plt.show()
#cv2.imwrite('houghlines1.jpg',img)

In [ ]:
#hough that uses less test points, less computations
img = cv2.imread('4-knee.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
minLineLength = 100
maxLineGap = 20
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
#print lines
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
plt.show()
#cv2.imwrite('houghlines2.jpg',img)